In [1]:
root = '../datasets/NCPPolicies/'
with open(root+'NCPPolicies_context_20200301.csv', 'r', encoding='utf-8') as file:
    context_lines = file.readlines()
context_lines = [line.strip().split('\t') for line in context_lines[1:]]
context_lines = {line[0]: '。'.join(line[1:]) for line in context_lines}
maxlen = max([len(each) for each in context_lines])
minlen = min([len(each) for each in context_lines])
# print(maxlen, minlen)
# for i in range(len(context_lines)):
#     if len(context_lines[i]) == 111:
#         print(i, context_lines[i])
with open(root+'NCPPolicies_train_20200301.csv', 'r', encoding='utf-8') as file:
    train_lines = file.readlines()
train_lines = [line.strip().split('\t') for line in train_lines[1:]]
maxlen = max([len(each) for each in train_lines])
minlen = min([len(each) for each in train_lines])
print(maxlen, minlen)

4 4


In [2]:
# 'fdgdsfasdfa'.join(['1232', '123', '123'])
data = []
train_lines = [[context_lines[each[1]], each[2], each[3]] for each in train_lines]
for i in range(len(train_lines)):
    start = train_lines[i][0].find(train_lines[i][2])
    if start != -1:
        data.append([train_lines[i][0], train_lines[i][1], [start, start+len(train_lines[i][2])]])

In [3]:
import public_params as pp
tokenizer = pp.tokenizer
data = [[tokenizer.convert_tokens_to_ids(list(each[0])), tokenizer.convert_tokens_to_ids(list(each[1])), each[2]] for each in data]

d:\install\Anaconda\envs\py3713\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def cut(data):

    q = [each[1] for each in data]
    c = [each[0] for each in data]
    labels = [each[2] for each in data]

    result = []
    maxlen4c = 330
    overlap = 20
    for i in range(len(data)):
        maxlen4c = 505 - len(q[i])
        if len(c[i]) > maxlen4c:
            for j in range(0, len(c[i])-overlap, maxlen4c-overlap):
                if labels[i][0]<j and labels[i][1]<j:
                    result.append([c[i][j: j+maxlen4c], q[i], [0, 0]])
                elif labels[i][0]<j and labels[i][1]>=j:
                    result.append([c[i][j: j+maxlen4c], q[i], [0, labels[i][1]-(j)]])
                elif labels[i][0]>=j and labels[i][1]<j+maxlen4c:
                    result.append([c[i][j: j+maxlen4c], q[i], [labels[i][0]-(j), labels[i][1]-(j)]])
                elif labels[i][0]<j+maxlen4c and labels[i][1]>=j+maxlen4c:
                    result.append([c[i][j: j+maxlen4c], q[i], [labels[i][0]-(j), maxlen4c-1]])
                elif labels[i][0]>=j+maxlen4c and labels[i][1]>=j+maxlen4c:
                    result.append([c[i][j: j+maxlen4c], q[i], [0, 0]])
                if j+maxlen4c >= len(c[i]):
                    break
        else: 
            result.append([c[i], q[i], labels[i]])
    return result

data = cut(data)

In [7]:
import numpy
numpy.save(f'./data4finetuning/NCPPolicies', data)

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1134, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 311, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "C:\Users\fzq19\AppData\Roaming\Python\Python37\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2062, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "C:\Users\fzq19\AppData\Roaming\Python\Python37\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2098, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [6]:
ml = max([len(each[0])+len(each[1]) for each in data])
ml

505